# **Cosine Similarity**

    두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도.
    
    0도(같은 방향)  : 1 / 90도 : 0 / 180도(반대 방향) : -1 
    
    수식 : 
$$ similarity = \cos(\theta) =  {A * B \over||A||||B||} = {\sum_{i=1}^n A_i * B_i \over{\sqrt{\sum_{i=1}^n(A_i)^2}} * {\sqrt{\sum_{i=1}^n(B_i)^2}}} $$
    
    문서 단어 행렬이나 TF-IDF 행렬을 통해서 문서 유사도를 구하는 경우 행렬이 각각의 특징 벡터 A,B가 된다.

### Numpy 이용한 구현

    문서1 : 저는 사과 좋아요
    문서2 : 저는 바나나 좋아요
    문서3 : 저는 바나나 좋아요 저는 바나나 좋아요

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

corpus =[
    "저는 사과 좋아요",
    "저는 바나나 좋아요",
    "저는 바나나 좋아요 저는 바나나 좋아요",
    ]

vectorizer = CountVectorizer()

DTM = vectorizer.fit_transform(corpus)
DTM = DTM.toarray()
print(DTM)

[[0 1 1 1]
 [1 0 1 1]
 [2 0 2 2]]


In [2]:
import numpy as np

def cos_sim(A,B) :
    return np.dot(A,B) / (np.linalg.norm(A) * np.linalg.norm(B))

print(cos_sim(DTM[0],DTM[1]))
print(cos_sim(DTM[1],DTM[2]))
print(cos_sim(DTM[2],DTM[0]))

0.6666666666666667
1.0000000000000002
0.6666666666666667


문서 1,2와 문서 1,3의 유사도가 같고, 문서 2,3의 유사도가 1이 나옴.

문서 2와 3의 유사도가 1. 

    문서의 길이가 다른 상황에서도 비교적 공정한 비교 할 수 있게 해줌
    by 벡터의 크기가 아니라 벡터의 방향에 초점을 두고 있기 때문이다.


## **유사도를 이용한 추천 시스템 구현**

    영화 데이터셋을 가지고 영화 추천 시스템 만들기.(TF-IDF , Cosine Similarity)
    
    데이터 셋 : https://www.kaggle.com/rounakbanik/the-movies-dataset

In [3]:
# 데이터 처리
import pandas as pd

df = pd.read_csv("movies_metadata.csv",low_memory=False)
data = df[['title','overview']]
data = data.fillna('')

In [4]:
# TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(data['overview'])
print(tfidf.shape)

(45466, 75827)


In [5]:
# Cosine Similarity
from sklearn.metrics.pairwise import linear_kernel
cos_sim = linear_kernel(tfidf,tfidf)


In [6]:
indices = pd.Series(data.index,index=data['title']).drop_duplicates()
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [7]:
def getRecommendation(title,cos_sim=cos_sim):
    
    idx = indices[title]
    # 유사도 구하기.
    sim_scores = list(enumerate(cos_sim[idx]))
    # 유사도 따라 정렬
    sim_scores = sorted(sim_scores,key=lambda x : x[1], reverse=True)
    
    sim_scores = sim_scores[1:11]
    
    movies_indices = [i[0] for i in sim_scores]
    
    return data['title'].iloc[movies_indices]

In [8]:
getRecommendation('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object